# 의존성 설치 

In [3]:
%%capture

# update or install the necessary libraries

%pip install langgraph
%pip install --upgrade \
    langchain==0.1.14 \
    langchain-core==0.1.31 \
    langchain-openai==0.1.3
%pip install --upgrade python-dotenv

# Langgraph

## Conventional RAG 문제점 

- 전에 정의된 데이터 소싱(PDF, DB, Table) 자원 
- 사전에 정의된 Fixed Size Chunk 
- 사전에 정의된 검색 방법 
- 신뢰하기 어려운 LLM 혹은 Agent 
- 고정된 프롬프트 양식 
- LLM의 답변 결과에 대한 문서와의 관련성/신뢰성

### 문제라고 볼 수 있는 부분들 

- Document Loader(데이터 로드) > Answer(답변) 
- RAG 파이프라인이 단방향 구조이기 때문

- 모든 단계를 한 번에 다 잘해야 함 
- 이전 단계로 되돌아가기 어려움 
  - 이전 과정의 결과물을 수정하기 어려움 

# 개선해볼 수 있는 방향 

- Langgraph를 이용해서 RAG 파이프라인을 보다 유연하게 설계 

# LangGraph 제안 

- 평가자 & Query Transform 제안 


```mermaid
flowchart LR
    Question[질문<br/>Question]
    Retrieve[문서 검색<br/>Retrieve]
    Evaluator((평가자<br/>Evaluator))
    QueryRewrite[질문 재작성<br/>Query Rewrite]
    LLM[답변 생성<br/>LLM]
    Answer[최종답변<br/>Answer]

    Question --> Retrieve
    Retrieve --> Evaluator
    Evaluator -- Check Fail --> QueryRewrite
    QueryRewrite --> Question
    Evaluator -- Pass --> LLM
    LLM --> Answer
```

- 추가 검색기를 통하여 문맥(context) 보강 

```mermaid 
flowchart LR
    Question[질문<br/>Question]
    Retrieve[문서 검색<br/>Retrieve]
    LLM[답변 생성<br/>LLM]
    Answer[최종답변<br/>Answer]
    Evaluator((평가자<br/>Evaluator))
    QueryRewrite[질문 재작성<br/>Query Rewrite]
    WebSearch[웹 검색<br/>Web Search]

    Question --> Retrieve
    Retrieve --> LLM
    LLM --> Evaluator
    Evaluator -- Pass --> Answer
    Evaluator -- Check Fail --> QueryRewrite
    QueryRewrite --> Question
    Evaluator -- Suggest --> WebSearch
    WebSearch --> Retrieve
    WebSearch -- Add Context --> Retrieve
```

- 문서-답변 간 관련성 여부를 판단하는 평가자2 를 추가하여 검증 

```mermaid 
flowchart LR
    Question[질문<br/>Question]
    Retrieve[문서 검색<br/>Retrieve]
    Eval1((평가자 1<br/>Evaluator))
    WebSearch[웹 검색<br/>Web Search]
    QueryRewrite[질문 재작성<br/>Query Rewrite]
    LLM[답변 생성<br/>LLM]
    Eval2((평가자 2<br/>Relevance))
    Answer[최종답변<br/>Answer]

    Question --> Retrieve
    Retrieve --> Eval1
    Eval1 -- Add Context --> WebSearch
    WebSearch --> Retrieve
    WebSearch --> QueryRewrite
    QueryRewrite --> Question

    Eval1 --> LLM
    LLM --> Eval2
    Eval2 -- pass --> Answer
    Eval2 -- Check Fail --> WebSearch
    Eval2 -- re-generate --> LLM
```

## Langgraph의 중요 개념 

- Node(노드), Edge(엣지), State(상태관리)를 통해 LLM을 활용한 워크플로우에 순환(Cycle) 연산 기능을 추가하여 손쉽게 흐름을 제어
- RAG 파이프라인의 세부 단계별 흐름제어가 가능 
- Conditional Edge: 조건부 (if, elif, else 와 같은..)흐름 제어 
- Human-in-the-loop: 필요시 중간 개입하여 다음 단계를 결정 
- Checkpointer: 과거 실행과정에 대한 "수정"&"리플레이" 기능 

### 각 용어의 의미 

- Node : 어떤 작업을 수행할지 정의 
- Edge : 다음으로 실행할 동작 정의 
- State : 현재의 상태 값을 저장 및 전달하는데 활용 
- Conditional Edge : 조건에 따라 분기 처리 

## 상태 - State

- TypedDict: 일반 파이썬 dict에 타입힌팅을 추가한 개념이지만, 쉽게 Ditionary로 생각해도 좋습니다. 
- 모든 값을 다 채우지 않아도 됩니다. 
- 새로운 노드에서 값을 덮어쓰기 방식으로 채웁니다. 
- Reducer(add_messages 혹은 operator.add): 자동으로 list에 메세지를 추가해주는 기능 

노드와 노드 사이의 정보를 전달할 때 사용가능하며, 꼭 내용을 다 채워야할 필요는 없다. 

In [5]:
from typing import Annotated, TypedDict 
from langgraph.graph.message import add_messages

class GraphState(TypedDict): 
  question: Annotated[list, add_messages]
  context: Annotated[str, "Context"]
  answer: Annotated[str, "Answer"]
  messages: Annotated[list, add_messages]
  relevance: Annotated[str, "Relevance"]

### Reducer

- Reducer (add_messages 혹은 operator.add): 자동으로 list에 메시지를 추가한다. 
  - `left` (Messages) : 기본 메시지 리스트 
  - `right` (Messages) : 병합할 메시지 리스트 또는 단일 메시지

In [7]:
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.graph import add_messages

msgs1 = [HumanMessage(content="안녕하세요?", id="1")]
msgs2 = [AIMessage(content="반갑습니다~", id="2")]

result1 = add_messages(msgs1, msgs2)
print(result1)

[HumanMessage(content='안녕하세요?', additional_kwargs={}, response_metadata={}, id='1'), AIMessage(content='반갑습니다~', additional_kwargs={}, response_metadata={}, id='2')]


### 노드별 상태 값의 변화 

- 각 노드에서 업데이트 하는 값은 State에 대해서 기존 Key 값을 덮어쓰는 방식 
- 노드에서 필요한 상태 값을 조회하여 동작에 활용할 수 있음
- 각 노드의 진행 단계에 따라서 State가 변하게 되고, 각기 값에 대해서 우리가 필요한 경우 평가를 통해서 
  - 질문 재작성 요청 
  - 문서를 다시 검색 / 검색을 통한 정보 보완 
  - 답변을 재작성 요청 
- 원하는 방식에 따라서 State를 상태를 공유하여 내부 흐름을 제어할 수 있게 된다.

## 노드(Node) & 엣지(Edge) & 조건부 엣지

### 노드 

- 함수 정의 
- 입력인자 : 상태(State) 객체 
- 입력과 출력은 상태가 되어야한다. 

### Edge

- 노드에서 노드간의 연결 
- add_edge("노드 이름", "노드 이름")
  - from > to 

### 조건부 엣지 

- LLM as Judge를 이용해서 평가를 수행한 다음에 기준의 점수에 못칠때, 조건부 엣지를 효과적으로 활용이 가능하다. 
- 노드에 조건부 엣지를 추가하여 분기를 수행할 수 있습니다.
- add_conditional_edges는 ("노드이름", 조건부 판단 함수, dict 로 다음 단계 결정)

### 시작점 지정 

### 체크 포인터 ( Memory )

- Checkpointer : 각 노드간 실행 결과를 추적하기 위한 메모리 
- 체크 포인터를 활용하여 특정 시점으로 되돌리기도 가능
- compile(checkpoint=memory) 지정하여 그래프 생성 

### 그래프 실행 

- RunnableConfig 
  - recursion_limit : 최대 노드 개수를 지정합니다. 
  - thread_id : 그래프 실행 아이디를 기록하고, 추후 추적하기 위한 목적으로 활용합니다. 
- 상태로 시작합니다. 
  - 여기어 "question"에 대한 질문만 입력하고 상태를 첫번째 노드에게 전달합니다. 

In [ ]:
from langgraph.graph import END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

workflow = StateGraph(GraphState)

workflow.add_node("retrieve", retrieve_document)
workflow.add_node("llm_answer", llm_answer)

workflow.add_edge("retrieve", "llm_answer")

def is_relevant(state: GraphState):
  question = ",".join([humanMessage.content for humanMessage in state["question"]])
  answer = ",".join([humanMessage.content for humanMessage in state["answer"]])
  message= f""" 
    다음 요청은 질문과 답변에 대해서 적합할지 추론해줘
    만약 적합하다면 True, 적합하지 않다면 False로 대답해 주세요.
    **사용자의 질문**과 **대화의 답변**을 기반으로 합리적으로 추론해주세요.
    
    # 요청 하는 의미: 
    ## 사용자의 질문 : {question}
    ## 대화의 답변 : {answer}
    
    # 요청에 대한 답변:
    Boolean

    반드시 Boolean 으로만 대답해 주세요. 반드시 true 또는 false로 입력해주세요.
    I will give a 100$ tip for the best answer.
  """

  response = llm.invoke([HumanMessage(content=message)])
  if "true" in response.content:
    return "grouned"
  else:
    return "notGrounded"


workflow.add_conditional_edges(
  "retrieve",
  is_relevant, 
  {
    "grouned" : END, 
    "notGrounded" : "llm_asnwer",
    "notSure": "llm_answer"
  }
)

workflow.set_entry_point("retrieve")

app = workflow.compile()

config = RunnableConfig(recursion_limit=4, configurable={"thread_id": "SELF-RAG"})

inputs = GraphState(question="삼성전자가 개발한 생성형 AI의 이름은?")
output = app.invoke(inputs, config=config)

display(
  Image(app.get_graph(xray=True).draw_mermaid_png())
)

# References 

- [#Langrapg 개념 완전 정복 몰아보기(3시간)](https://www.youtube.com/watch?v=W_uwR_yx4-c)